# Skyrim Wiki Scraper

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
# Function to scrape the wiki data
def get_information(full_path):

  r = requests.get(full_path)
  # Convert to beautiful soup object
  soup = bs(r.content)
  # Print out the HTML
  contents = soup.prettify()
  info_box = soup.find(class_='wikitable infobox')
  info_rows = info_box.find_all('tr')

  character_info = {}

  for index, row in enumerate(info_rows):
    character_info['Source'] = full_path
    if index == 0:
      character_info['Name'] = row.find('th').get_text().split('\n')[0]
    elif row.find('td') == None:
      content_key = row.find('th').get_text()
      character_info[content_key] = True  
    else:
      if (len(row.find_all('th')) and len(row.find_all('td'))) > 1:
        content_keys = row.find_all('th')
        content_values = row.find_all('td')
        for key, value in zip(content_keys, content_values):
          character_info[key.get_text()] = value.get_text()

      elif row.find_all('img'):
        content_key = row.find('th').get_text()
        content_value = ''
        for i, img in enumerate(row.select('img')):
          if i == 1:
            content_value=img['alt']
        character_info[content_key] = content_value

      elif len(row.select('td a')) > 1:
        content_key = row.find('th').get_text()
        content_value = row.find('td').get_text().replace('\n', ', ')
        character_info[content_key] = content_value

      else:
        content_key = row.find('th').get_text()
        content_value = row.find('td').get_text()
        character_info[content_key] = content_value
  return(character_info)

In [3]:
# Get the list of the links for each characters
r = requests.get('https://en.uesp.net/wiki/Skyrim:People')
soup = bs(r.content)
people_list = soup.select('.wikitable tr b > a')

In [4]:
# Scrape the data based on the links
characters_data = []
base_link = 'https://en.uesp.net'

for index, people in enumerate(people_list):
  try:
    relative_link = people['href']
    fullpath = base_link + relative_link
    characters_data.append(get_information(fullpath))
  except Exception as e:
    print('WARNING: Failed to take ' + people.get_text() + ' ' + fullpath)
len(characters_data)

1051

In [1]:
# Function to save and load the JSON file
import json

def save_data(title, data):
  with open(title, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

def load_data(title):
  with open(title, encoding='utf-8') as f:
    return json.load(f)